Notebook to process abf files into dictionaries with waveforms

In [ ]:
import pandas as pd

Code to process excel spreadsheet into readable data frame with 'Trace name' as the index. Just download csv tab from spreadsheet and run

In [ ]:
def load_and_clean_csv(filepath):
    # Read the raw CSV , headers are irregular
    df_raw = pd.read_csv(filepath, header=None)

    # The actual data starts from the 2nd row (index 1), so slice from there
    data = df_raw.iloc[1:]

    # Each block has 5 columns: Trace name, ID, On time, Freq, Tags
    num_cols = df_raw.shape[1]
    block_size = 5

    # Find how many full 5-column blocks exist
    blocks = []
    for start in range(1, num_cols, block_size + 1):  # +1 accounts for those extra separators
        end = start + block_size
        if end <= num_cols:
            block = data.iloc[:, start:end]
            block.columns = ['Trace name', 'ID', 'On time', 'Frequency', 'Swim description']
            blocks.append(block)

    # Combine all blocks into one DataFrame
    df_clean = pd.concat(blocks, ignore_index=True)

    # Drop rows that are completely empty 
    df_clean = df_clean.dropna(how='all')

    # Set index to trace name and delete first row with column labels
    df_clean.set_index('Trace name',inplace=True)
    df_clean = df_clean.iloc[1:]
    return df_clean


Access specific abf file using df['trace_name']

Function that splits abf df into dictionary with each waveworm, just load abf file and then pass cleaned df that goes with abf file

In [ ]:
def make_waveforms(abf,df):
    '''
    Function that takes an abf file and a df of the annotations 
    Returns a dictionary with waveforms labeled by their frequency
    '''
    abf.setSweep(sweepNumber=0, channel=0)
    x = abf.sweepX
    y = abf.sweepY
    abf_df = pd.DataFrame({
        'Time': x,
        'Current': y
    })   
    df["Seconds"] = df['On time']*0.001
    waveforms = {} 
    for i in range(len(df) - 1):
        t_0 = df.loc[i, 'Seconds']
        t_f = df.loc[i + 1, 'Seconds']
        phase_0 = int(t_0 * 100000)
        phase_1 = int(t_f * 100000)
        abf_waveform = abf_df.loc[phase_0:phase_1].copy()
        abf_waveform["Phase"] = (abf_waveform["Time"] - t_0) / (t_f - t_0)
        freq = df.loc[i + 1, "Freq"]
        waveforms[freq] = abf_waveform

    return waveforms


Same function with include tags option

In [ ]:
def make_waveforms(abf, df, include_tags=False):
    '''
    Function that takes an abf file and a df of the annotations.
    Returns a dictionary with waveforms labeled by their frequency.
    
    If include_tags=True, each entry includes the waveform DataFrame and its tags.
    '''
    abf.setSweep(sweepNumber=0, channel=0)
    x = abf.sweepX
    y = abf.sweepY
    abf_df = pd.DataFrame({
        'Time': x,
        'Current': y
    })   
    df["Seconds"] = df['On time'] * 0.001
    waveforms = {}

    for i in range(len(df) - 1):
        t_0 = df.loc[i, 'Seconds']
        t_f = df.loc[i + 1, 'Seconds']
        phase_0 = int(t_0 * 100000)
        phase_1 = int(t_f * 100000)
        abf_waveform = abf_df.loc[phase_0:phase_1].copy()
        abf_waveform["Phase"] = (abf_waveform["Time"] - t_0) / (t_f - t_0)
        freq = df.loc[i + 1, "Freq"]

        if include_tags:
            tags_current = df.loc[i, "Tags"] if isinstance(df.loc[i, "Tags"], str) else ""
            tags_next = df.loc[i + 1, "Tags"] if isinstance(df.loc[i + 1, "Tags"], str) else ""
            waveforms[freq] = {
                "waveform": abf_waveform[["Time", "Current", "Phase"]],
                "tags_current": tags_current,
                "tags_next": tags_next
            }
        else:
            waveforms[freq] = abf_waveform[["Time", "Current", "Phase"]]

    return waveforms


In [ ]:
dict = make_waveforms(abf, df, include_tags=False)


Waveforms labeled by frequency:

In [ ]:
dict.keys()

Get one wave using key

In [ ]:
onewave = dict[np.float64(34.0716)]
print(onewave)

Bin data

In [ ]:

# Make edges for the bin
bins = np.linspace(0, 1, 51, endpoint = True)  
# print(bins)

# Cut the phase into bins
onewave['Phase Bin'] = pd.cut(onewave['Phase'], bins=bins, include_lowest=True)
# print(onewave)

# Group by the bins and calculate the average current
# binned_avg = onewave.groupby(by='Phase Bin','Current')
binned_avg = onewave.groupby('Phase Bin')['Current'].mean().reset_index()
# print(binned_avg)

# Get phase column for plotting (middle of bin)
phase = np.arange(0.01, 1.00001, 0.02)
# len(phase)
binned_avg['Phase'] = phase
binned_avg.head()

Plot the waveform"

In [ ]:
binned_avg.plot('Phase','Current',kind = 'line')